In [1]:
import datetime
import numpy as np
import os
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
#Created a 01/27/2022 
print(f'Last edited: {datetime.datetime.now()}')

Last edited: 2022-02-03 10:41:30.994776


In [2]:
### load the data 
DataDict = np.load('ProDataDict.npy', allow_pickle=True)
DataDict = DataDict.item()
for key in DataDict:
    print(key)
    
    print(f'Data shape = {DataDict[key].shape}')

10pM
Data shape = (61, 301)
15pM
Data shape = (61, 300)
20pM
Data shape = (61, 300)
50pM
Data shape = (61, 300)


In [ ]:
# following block saves means, and <lnx> to be used as constraints

In [29]:
const_dict = defaultdict(lambda:'not present')
time_values = [0,6,12,24,45,60,90]
times_constrained = np.array(time_values) #minutes 
idx_list = np.array(times_constrained/3, dtype=int)
muFull = np.zeros(len(time_values)*len(DataDict))
lnFull = np.zeros(len(time_values)*len(DataDict))
i=0

for key in DataDict:
    # relevant data
    drel  = DataDict[key][idx_list,:]
    muarr = np.mean(drel, axis = 1)
    muFull[i:i+len(muarr)] = muarr
    drel  = np.log(DataDict[key][idx_list,:])
    lnarr = np.mean(drel, axis = 1)
    lnFull[i:i+len(muarr)] = lnarr

    i+= len(muarr)

        


In [46]:
# following block saves fraction of cells in the left and the right tail of the distribution


In [26]:
farr = np.ones(len(times_constrained)*len(DataDict)*2)*.25
# print(farr.shape)
consarr = np.concatenate((muFull, lnFull,farr ), axis =0)

In [ ]:
# following block saves labels array 

In [67]:
## make the labels of the cons array 
mulabels = []
lnlabels = []
flabels = []
for key in DataDict:
    for t in time_values:
        mulabels.append(r'mu , ' + f'{key}, {t} min' )
        lnlabels.append('<ln(x)> , ' + f'{key}, {t} min')
        flabels.append('fl , ' + f'{key}, {t} min')
#         flabels.append('fr , ' + f'{key}, {t} min')

for key in DataDict:
    for t in time_values:
        flabels.append('fr , ' + f'{key}, {t} min')
        
# print(mulabels, '\n',lnlabels , '\n',flabels  )
labels = mulabels + lnlabels + flabels

In [68]:
flabels

['fl , 10pM, 0 min',
 'fl , 10pM, 6 min',
 'fl , 10pM, 12 min',
 'fl , 10pM, 24 min',
 'fl , 10pM, 45 min',
 'fl , 10pM, 60 min',
 'fl , 10pM, 90 min',
 'fl , 15pM, 0 min',
 'fl , 15pM, 6 min',
 'fl , 15pM, 12 min',
 'fl , 15pM, 24 min',
 'fl , 15pM, 45 min',
 'fl , 15pM, 60 min',
 'fl , 15pM, 90 min',
 'fl , 20pM, 0 min',
 'fl , 20pM, 6 min',
 'fl , 20pM, 12 min',
 'fl , 20pM, 24 min',
 'fl , 20pM, 45 min',
 'fl , 20pM, 60 min',
 'fl , 20pM, 90 min',
 'fl , 50pM, 0 min',
 'fl , 50pM, 6 min',
 'fl , 50pM, 12 min',
 'fl , 50pM, 24 min',
 'fl , 50pM, 45 min',
 'fl , 50pM, 60 min',
 'fl , 50pM, 90 min',
 'fr , 10pM, 0 min',
 'fr , 10pM, 6 min',
 'fr , 10pM, 12 min',
 'fr , 10pM, 24 min',
 'fr , 10pM, 45 min',
 'fr , 10pM, 60 min',
 'fr , 10pM, 90 min',
 'fr , 15pM, 0 min',
 'fr , 15pM, 6 min',
 'fr , 15pM, 12 min',
 'fr , 15pM, 24 min',
 'fr , 15pM, 45 min',
 'fr , 15pM, 60 min',
 'fr , 15pM, 90 min',
 'fr , 20pM, 0 min',
 'fr , 20pM, 6 min',
 'fr , 20pM, 12 min',
 'fr , 20pM, 24 min',
 '

In [69]:
## save the constraints dictionary 
consdict = {'array': consarr, 'labels': labels}
np.save('Arrays_for_max_ent/cons_dict_mu_lnx_fraclr_020322.npy', consdict,allow_pickle='TRUE')
# real_cons = read_dictionary['array']

In [51]:
d = np.load('Arrays_for_max_ent/cons_dict_mu_lnx_fraclr_020322.npy', allow_pickle=True ).item()
len(d['array'])

112

In [52]:
28*2 + 28*2

112

In [73]:
# The constraints of choice will be for each condition in 
# time_values = [0,6,12,24,45,60,90] 
# and ligand concentration [10,15,20,50]pM 
# save means and lnx and the number 
# save the x limits at which those numbers happen of cells in 25 percentile and 75 percentile 
time_values = [0,6,12,24,45,60,90]
times_constrained = np.array(time_values) #minutes 
idx_list = np.array(times_constrained/3, dtype=int)
XLimDict = defaultdict(lambda:"not present")
## do it with one key first 
for key in DataDict:
    data = DataDict[key]
#     data.shape
    ChosenData = data[idx_list]
    x25, x75 = np.percentile(ChosenData, [25,75], axis = 1)
    if key == '10pM': 
        conc25 = x25
        conc75 = x75
    else:
        conc25 = np.concatenate((conc25, x25), axis = 0)
        conc75 = np.concatenate((conc75, x75), axis = 0)

    
    XLimDict[f'{key}_x25'] = x25
    XLimDict[f'{key}_x75'] = x75
#     print(x25, x75)
#     np.concatenate((x25,x75), axis = 0)
XLimDict

defaultdict(<function __main__.<lambda>()>,
            {'10pM_x25': array([420.74037322, 406.97179005, 404.67064872, 386.71511377,
                    357.66530152, 347.3870103 , 346.36910602]),
             '10pM_x75': array([519.9828141 , 492.68499047, 494.79555799, 471.86109448,
                    454.05063572, 442.91101087, 442.18722051]),
             '15pM_x25': array([426.98937396, 414.22918139, 403.28844357, 362.1294209 ,
                    322.22365368, 301.3992775 , 297.89190894]),
             '15pM_x75': array([521.57326038, 510.15233656, 494.09086526, 456.02003879,
                    426.00910426, 412.35375495, 413.54695713]),
             '20pM_x25': array([420.73364085, 391.36126366, 383.54023424, 330.29628056,
                    279.58182666, 266.99264032, 263.28569446]),
             '20pM_x75': array([517.04544656, 483.43467766, 470.7292563 , 422.59028   ,
                    377.31168165, 363.52503148, 360.9157608 ]),
             '50pM_x25': array([417.15413435

In [77]:
conc25.shape


(28,)

In [76]:
labelsxlim = []
for key in DataDict:
    for t in time_values:
        labelsxlim.append(f'{key}, {t} min')
labelsxlim

['10pM, 0 min',
 '10pM, 6 min',
 '10pM, 12 min',
 '10pM, 24 min',
 '10pM, 45 min',
 '10pM, 60 min',
 '10pM, 90 min',
 '15pM, 0 min',
 '15pM, 6 min',
 '15pM, 12 min',
 '15pM, 24 min',
 '15pM, 45 min',
 '15pM, 60 min',
 '15pM, 90 min',
 '20pM, 0 min',
 '20pM, 6 min',
 '20pM, 12 min',
 '20pM, 24 min',
 '20pM, 45 min',
 '20pM, 60 min',
 '20pM, 90 min',
 '50pM, 0 min',
 '50pM, 6 min',
 '50pM, 12 min',
 '50pM, 24 min',
 '50pM, 45 min',
 '50pM, 60 min',
 '50pM, 90 min']

In [78]:
XLimitsDict_new = {'labels': labelsxlim, 'x25': conc25, 'x75': conc75 } 
np.save('Arrays_for_max_ent/XLimDict_25_75_020322.npy' ,XLimitsDict_new)

In [79]:
XLimitsDict_new

{'labels': ['10pM, 0 min',
  '10pM, 6 min',
  '10pM, 12 min',
  '10pM, 24 min',
  '10pM, 45 min',
  '10pM, 60 min',
  '10pM, 90 min',
  '15pM, 0 min',
  '15pM, 6 min',
  '15pM, 12 min',
  '15pM, 24 min',
  '15pM, 45 min',
  '15pM, 60 min',
  '15pM, 90 min',
  '20pM, 0 min',
  '20pM, 6 min',
  '20pM, 12 min',
  '20pM, 24 min',
  '20pM, 45 min',
  '20pM, 60 min',
  '20pM, 90 min',
  '50pM, 0 min',
  '50pM, 6 min',
  '50pM, 12 min',
  '50pM, 24 min',
  '50pM, 45 min',
  '50pM, 60 min',
  '50pM, 90 min'],
 'x25': array([420.74037322, 406.97179005, 404.67064872, 386.71511377,
        357.66530152, 347.3870103 , 346.36910602, 426.98937396,
        414.22918139, 403.28844357, 362.1294209 , 322.22365368,
        301.3992775 , 297.89190894, 420.73364085, 391.36126366,
        383.54023424, 330.29628056, 279.58182666, 266.99264032,
        263.28569446, 417.15413435, 381.71040546, 338.20705474,
        227.71067472, 178.13591228, 174.50144449, 179.6756887 ]),
 'x75': array([519.9828141 , 492.684

In [70]:
conclist = ['10pM' , '15pM' , '20pM' , '25pM']
# print(XLimDict)
i = 0 
x25list =
x72list = 
for conc in conclist:
#     if i==0:
        concarr_25 = XLimDict[key]
#     else:
        concarr_25 = np.concatenate(concarr,XLimDict[key] )
    i+=1 

    
print(concarr)

{'10pM_x25': array([420.74037322, 406.97179005, 404.67064872, 386.71511377,
        357.66530152, 347.3870103 , 346.36910602]),
 '10pM_x75': array([519.9828141 , 492.68499047, 494.79555799, 471.86109448,
        454.05063572, 442.91101087, 442.18722051]),
 '15pM_x25': array([426.98937396, 414.22918139, 403.28844357, 362.1294209 ,
        322.22365368, 301.3992775 , 297.89190894]),
 '15pM_x75': array([521.57326038, 510.15233656, 494.09086526, 456.02003879,
        426.00910426, 412.35375495, 413.54695713]),
 '20pM_x25': array([420.73364085, 391.36126366, 383.54023424, 330.29628056,
        279.58182666, 266.99264032, 263.28569446]),
 '20pM_x75': array([517.04544656, 483.43467766, 470.7292563 , 422.59028   ,
        377.31168165, 363.52503148, 360.9157608 ]),
 '50pM_x25': array([417.15413435, 381.71040546, 338.20705474, 227.71067472,
        178.13591228, 174.50144449, 179.6756887 ]),
 '50pM_x75': array([527.47836768, 481.53582215, 433.31089256, 319.58976346,
        251.12868553, 247.35

In [65]:
## check that actually those lims are correct 
# xlimdict2 
for key in DataDict: 
    data = DataDict[key]
    ChosenData = data[idx_list]
    for ti in range(len(time_values)):
        x25 = XLimDict[f'{key}_x25'][ti]
        x75 = XLimDict[f'{key}_x75'][ti]
        d = ChosenData[ti,:]
        print(len(np.where(d>x75)[0])/len(d))
    
    
    # that works so let's save !!! 

0.24916943521594684
0.24916943521594684
0.24916943521594684
0.24916943521594684
0.24916943521594684
0.24916943521594684
0.24916943521594684
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25


In [66]:
XLimDict = dict(XLimDict)
np.save('Arrays_for_max_ent/XLimDict_25_75_020322.npy' ,XLimDict)